In [17]:
import pickle
import pandas as pd
import datetime


In [12]:
with open('./submission_data/2_ts_prediction/all_preds', 'rb') as f:
    array = pickle.load(f)
    
with open('./submission_data/2_ts_prediction/mse_after', 'rb') as f:
    mse_after = pickle.load(f)

with open('./submission_data/2_ts_prediction/mse_before', 'rb') as f:
    mse_before = pickle.load(f)

In [137]:
sample_sub_1 = pd.read_csv('./sample_submissions/sample_sub_2.csv')
sample_sub_1

,year,station_id,month,day,date,delta_stage_max
0,1993,3019,4,111,1993-04-21,0
1,1993,3019,4,112,1993-04-22,0
2,1993,3019,4,113,1993-04-23,0
3,1993,3019,4,114,1993-04-24,0
4,1993,3019,4,115,1993-04-25,0
...,...,...,...,...,...,...
1234,2013,3230,5,134,2013-05-14,0
1235,2013,3230,5,135,2013-05-15,0
1236,2013,3230,5,136,2013-05-16,0
1237,2013,3230,5,137,2013-05-17,0


In [138]:
sample_sub_2 = pd.read_csv('./sample_submissions/sample_sub_2.csv')
sample_sub_2['date'] = pd.to_datetime(sample_sub_2['date'])
sample_sub_2.drop(['delta_stage_max', 'year', 'day', 'month'], axis=1, inplace=True)
sample_sub_2

,station_id,date
0,3019,1993-04-21
1,3019,1993-04-22
2,3019,1993-04-23
3,3019,1993-04-24
4,3019,1993-04-25
...,...,...
1234,3230,2013-05-14
1235,3230,2013-05-15
1236,3230,2013-05-16
1237,3230,2013-05-17


In [139]:
def create_submission(preds, mse, mse_tune):
    df = pd.DataFrame(columns=['year', 'station_id', 'month', 'day', 'date', 'delta_stage_max'])
    
    for index, row in enumerate(preds):
        new_row = {}
        date = pd.to_datetime(row[0])
        station_id = row[1]
        
        new_row['station_id'] = station_id
        new_row['year'] = date.year
        new_row['month'] = date.month
        new_row['day'] = date.timetuple().tm_yday
        
        if mse_tune[index] > mse[index]:
            predictions = row[3]
        else:
            predictions = row[2]
            
        for i, val in enumerate(predictions):
            if i == 0:
                new_row['delta_stage_max'] = predictions[i + 1] - predictions[i]
            else:
                new_row['delta_stage_max'] = predictions[i] - predictions[i - 1]
            
            new_row['stage_max_before'] = row[2][i]
            new_row['stage_max_after'] = row[3][i]
            new_row['stage_max'] = val
            new_row['date'] = date + datetime.timedelta(days=i)
            new_row['day'] = (date + datetime.timedelta(days=i)).timetuple().tm_yday
            new_row['mse_after'] = mse[index]
            new_row['mse_before'] = mse_tune[index]
            new_row['mse_diff'] = mse_tune[index] - mse[index]
            
            df = df.append(new_row, ignore_index=True)
            
    
            
    return df
            

In [149]:
results = create_submission(array, mse_after, mse_before)
results = results.iloc[:,:-6]
results.head()

,year,station_id,month,day,date,delta_stage_max
0,1993,3230,4,111,1993-04-21,3.663166
1,1993,3230,4,112,1993-04-22,3.663166
2,1993,3230,4,113,1993-04-23,4.459302
3,1993,3230,4,114,1993-04-24,6.484208
4,1993,3230,4,115,1993-04-25,7.063216


In [141]:
results = pd.merge(sample_sub_2, results, how="inner", on=["station_id", "date"])
results = results.reindex(columns=['year', 'station_id', 'month', 'day', 'date', 'delta_stage_max'])
results.head()

,year,station_id,month,day,date,delta_stage_max
0,1993,3019,4,111,1993-04-21,1.282488
1,1993,3019,4,112,1993-04-22,1.282488
2,1993,3019,4,113,1993-04-23,1.278191
3,1993,3019,4,114,1993-04-24,1.176855
4,1993,3019,4,115,1993-04-25,1.572330


In [144]:
results.to_csv('submission_2.csv', index=False)

In [148]:
sample_sub_1['delta_stage_max'] = pd.read_csv('submission_2.csv')['delta_stage_max']
sample_sub_1.to_csv('submission_2.csv', index=False)